<a href="https://colab.research.google.com/github/Pavansree14/Python/blob/Pyspark_JSON_metadata_analysis/tes1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip3 install pyspark

In [30]:
from pyspark import SparkContext, SparkConf

In [31]:
conf = SparkConf().setAppName("KDDCup_PySpark").setMaster("local[*]")
sc = SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=KDDCup_PySpark, master=local[*]) created by __init__ at <ipython-input-4-bd35e2a7766c>:2 

In [32]:
print(sc)

<SparkContext master=local[*] appName=KDDCup_PySpark>


In [6]:
import random
randomlist  = random.sample(range(0,40),10)
print(randomlist)

[15, 34, 10, 18, 39, 36, 26, 21, 3, 4]


In [16]:
rdd1 = sc.parallelize(randomlist, 4)

rdd1.collect()

[15, 34, 10, 18, 39, 36, 26, 21, 3, 4]

In [ ]:
rdd1.getNumPartitions()

4

In [ ]:
print(rdd1.glom().collect())

[[8, 29], [25, 24], [3, 38], [26, 37, 22, 28]]


In [ ]:
rdd_map = rdd1.map(lambda item: (item + 1)*3)
rdd_map.collect()

[27, 90, 78, 75, 12, 117, 81, 114, 69, 87]

In [ ]:
print(rdd_map.glom().collect())

[[27, 90], [78, 75], [12, 117], [81, 114, 69, 87]]


In [ ]:
rdd_filter = rdd1.filter(lambda x: x%3 == 0)
rdd_filter.collect()

[24, 3]

In [ ]:
(rdd_filter.glom().collect())

[[], [24], [3], []]

In [ ]:
#union
print(rdd1.collect())

rdd2 = sc.parallelize([14,20,28,30,24,86,32,22],2)
rdd2.collect()
rdd_union = rdd1.union(rdd2)
print(rdd_union.collect())

print(rdd_union.getNumPartitions())

[8, 29, 25, 24, 3, 38, 26, 37, 22, 28]
[8, 29, 25, 24, 3, 38, 26, 37, 22, 28, 14, 20, 28, 30, 24, 86, 32, 22]
6


In [ ]:
#Intersection
rdd_intersection = rdd1.intersection(rdd2)
print(rdd_intersection.collect())
print(rdd_intersection.getNumPartitions())
rdd_intersection.glom().collect()

[24, 22, 28]
6


[[24], [], [], [], [22, 28], []]

In [ ]:
#find empty partitions
counter = 0
for item in rdd_intersection.glom().collect():
    if len(item) == 0:
      counter = counter + 1

counter


4

In [ ]:
#coalesce(num of partitions)--reduce the size of partitions
rdd_intersection.coalesce(1).glom().collect()

[[24, 22, 28]]

In [ ]:
#takesample(withReplacement, num, [seed])
rdd1.takeSample(False, 5)

[28, 37, 3, 24, 8]

In [ ]:
#takeordered(n, [ordering])
print(rdd1.takeOrdered(5))
print(rdd1.takeOrdered(5, key = lambda x: -x))

[3, 8, 22, 24, 25]
[38, 37, 29, 28, 26]


In [ ]:
rdd2.glom().collect()

[[14, 20, 28, 30], [24, 86, 32, 22]]

In [ ]:
#reduce by key
rdd_rbk = sc.parallelize([(2,6),(7,5),(3,8),(6.8),(2,9),(5,3),(4,6),(6,9)], 2)

print(rdd_rbk.glom().collect())


[[(2, 6), (7, 5), (3, 8), 6.8], [(2, 9), (5, 3), (4, 6), (6, 9)]]


In [ ]:
#reduce
rdd1.reduce(lambda x,y: x-y)

74

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Read and Load Data to Spark
import json

rdd_json = sc.textFile("/content/drive/MyDrive/Colab Notebooks/arxiv-metadata-oai-snapshot.json",100)
rdd = rdd_json.map(lambda x: json.loads(x))
rdd.persist()

PythonRDD[2] at RDD at PythonRDD.scala:53

Query optimization
catalyst query optimization
Project tungsten.

In [ ]:
#Count of elements
rdd.count()

2011231

In [ ]:
#First two records
rdd.take(2)

[{'id': '0704.0001',
  'submitter': 'Pavel Nadolsky',
  'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan",
  'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies',
  'comments': '37 pages, 15 figures; published version',
  'journal-ref': 'Phys.Rev.D76:013009,2007',
  'doi': '10.1103/PhysRevD.76.013009',
  'report-no': 'ANL-HEP-PR-07-12',
  'categories': 'hep-ph',
  'license': None,
  'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with d

In [ ]:
# How to get all the distinct attributes
rdd.flatMap(lambda x: x.keys()).distinct().collect()

['authors',
 'comments',
 'title',
 'id',
 'journal-ref',
 'versions',
 'submitter',
 'categories',
 'update_date',
 'authors_parsed',
 'report-no',
 'license',
 'abstract',
 'doi']

In [ ]:
#Get the name of the license
rdd.map(lambda x: x["license"]).distinct().collect()

[None,
 'http://creativecommons.org/licenses/publicdomain/',
 'http://creativecommons.org/licenses/by-nc-nd/4.0/',
 'http://creativecommons.org/licenses/by-nc-sa/4.0/',
 'http://creativecommons.org/licenses/by-nc-sa/3.0/',
 'http://creativecommons.org/licenses/by/3.0/',
 'http://creativecommons.org/licenses/by/4.0/',
 'http://creativecommons.org/publicdomain/zero/1.0/',
 'http://arxiv.org/licenses/nonexclusive-distrib/1.0/',
 'http://creativecommons.org/licenses/by-sa/4.0/']

In [ ]:
shortest_tittle_rdd = rdd.map(lambda x: x["title"]).reduce(lambda x,y: x if x < y else y)
longest_tittle_rdd = rdd.map(lambda x: x["title"]).reduce(lambda x,y: x if x > y else y)

print("the shortest: ", shortest_tittle_rdd)
print("the longest: ", longest_tittle_rdd)


the shortest:  !-Graphs with Trivial Overlap are Context-Free
the longest:  Weyl formula for the negative dissipative eigenvalues of Maxwell's
  equations


In [ ]:
# Finding abbrevations with 5 or more letters in the abstract
import re

def get_abbrivations(line):
  result = re.search(r"\(([A-Za-z][^_/\\<>]{5,})\)", line)
  if result:
      return result.group(1)

In [ ]:
rdd.filter(lambda x: get_abbrivations(x['abstract'])).count()

702869

In [ ]:
rdd.filter(lambda x: get_abbrivations(x['abstract'])).take(5)

[{'id': '0704.0001',
  'submitter': 'Pavel Nadolsky',
  'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan",
  'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies',
  'comments': '37 pages, 15 figures; published version',
  'journal-ref': 'Phys.Rev.D76:013009,2007',
  'doi': '10.1103/PhysRevD.76.013009',
  'report-no': 'ANL-HEP-PR-07-12',
  'categories': 'hep-ph',
  'license': None,
  'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with d

In [ ]:
# Get the number of archive records per month(update_date' atrribute)
import datetime

def extract_month(MonthIn):
    d = datetime.datetime.strptime(MonthIn, "%Y-%m-%d")
    return d.month

extract_month('2003-12-13')

12

In [ ]:
rdd.map(lambda x: (extract_month(x['update_date']),1)).reduceByKey(lambda x,y: x+y).collect()

[(1, 134247),
 (2, 116948),
 (3, 126458),
 (4, 117126),
 (5, 296587),
 (6, 191746),
 (7, 122649),
 (8, 138469),
 (9, 138978),
 (10, 197755),
 (11, 297963),
 (12, 132305)]

In [ ]:
# Get the average number of pages

#import re
def get_Page(line):
    search = re.findall('\d+ pages', line)
    if search:
       return int(search[0].split(" ")[0])
    else:
       return 0

In [ ]:
rdd_average = rdd.map(lambda x: get_Page(x['comments'] if x['comments'] != None else "None"))

#remove 0
rdd_average = rdd_average.filter(lambda x: x!=0)

average_counter = rdd_average.count()
average_summation = rdd_average.reduce(lambda x,y: x+y)

print(average_counter)
print(average_summation)
print("the average of pages is", (average_summation/average_counter))

1184075
21139516
the average of pages is 17.85319004286046


# **Processing Large structured data using RDD**

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
#Read and load data to spark
rdd = sc.textFile("/content/drive/MyDrive/kddcup.data.gz")

In [15]:
# Get number of partitions
rdd = rdd.repartition(10)  #shuffle all data
print(sc.defaultParallelism)
print(rdd.getNumPartitions())

rdd.persist()

2
10


MapPartitionsRDD[12] at coalesce at NativeMethodAccessorImpl.java:0

In [8]:
#Get ten records randomly
rdd.takeSample(False, 10, 1234)

['0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '1,tcp,smtp,SF,1272,364,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,3,0.00,0.00,0.00,0.00,1.00,0.00,1.00,231,124,0.54,0.03,0.00,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,510,510,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,509,509,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,508,508,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,

In [13]:
# count elements
rdd.count()

4898431

In [18]:
#Calculate ratio of normal calculations
Normal_rdd = rdd.filter(lambda line: 'normal.' in line)

ratio =  (Normal_rdd.count()/rdd.count())*100

print("the ratio of normal connections is {} %".format(round(ratio),2))


the ratio of normal connections is 20 %


In [20]:
# Get the list of labels
Split_rdd = rdd.map(lambda line: line.split(","))
Label_rdd = Split_rdd.map(lambda item: item[-1]).distinct()
Label_rdd.collect()

['pod.',
 'neptune.',
 'warezclient.',
 'loadmodule.',
 'smurf.',
 'nmap.',
 'spy.',
 'back.',
 'teardrop.',
 'ipsweep.',
 'phf.',
 'ftp_write.',
 'multihop.',
 'guess_passwd.',
 'normal.',
 'imap.',
 'satan.',
 'land.',
 'portsweep.',
 'warezmaster.',
 'rootkit.',
 'perl.',
 'buffer_overflow.']

In [21]:
# Count of connections for each label
def LabelCount_func(items):
  Labels_Count = []
  for i in items:
    Labels_Count.append(rdd.filter(lambda line: i in line).count())

In [22]:
LabelCount_func(Label_rdd.collect())

In [23]:
# 2nd method in solution
%%time
Label_rdd_kv = Split_rdd.map(lambda x:(x[-1],1))
Label_rdd_Reduce = Label_rdd_kv.reduceByKey(lambda a,b: a+b)


CPU times: user 7.45 ms, sys: 1.71 ms, total: 9.15 ms
Wall time: 27.3 ms


In [26]:
import pandas as pd

keys = Label_rdd_Reduce.keys().collect()
values = Label_rdd_Reduce.values().collect()

DF_labels_kv = pd.DataFrame({'Label': keys,
                             'Count': values})

DF_labels_kv.sort_values(by="Count", ascending=False)

,Label,Count
4,smurf.,2807886
1,neptune.,1072017
14,normal.,972781
16,satan.,15892
9,ipsweep.,12481
18,portsweep.,10413
5,nmap.,2316
7,back.,2203
2,warezclient.,1020
8,teardrop.,979


In [35]:
Split_rdd.filter(lambda x: x[13] == '1')\
         .map(lambda x: (x[1], x[4], x[5]))\
         .filter(lambda x: int(x[2])> int(x[1])*500)\
         .collect()

[('tcp', '296', '507534'),
 ('tcp', '296', '507534'),
 ('tcp', '246', '866032'),
 ('tcp', '317', '394616'),
 ('tcp', '262', '744605'),
 ('tcp', '173', '744605'),
 ('tcp', '224', '2776333'),
 ('tcp', '262', '744605'),
 ('tcp', '0', '2072'),
 ('tcp', '351', '759161'),
 ('tcp', '1794', '3851730'),
 ('tcp', '465', '320362'),
 ('tcp', '0', '2072'),
 ('tcp', '0', '2072'),
 ('tcp', '296', '507534'),
 ('tcp', '266', '507534'),
 ('tcp', '255', '574784'),
 ('tcp', '0', '2072'),
 ('tcp', '353', '759161'),
 ('tcp', '433', '1524348')]

In [36]:
# Get the list of protocols that are vulnerable to attacks, where there is NOT guest login to the destination addresses

normal_protocols_rdd = Split_rdd.filter(lambda line: "normal" in line[-1] and line[21] !='1')\
                                .map(lambda line: (line[1], 1)).reduceByKey(lambda x,y: x+y)

attack_protocols_rdd = Split_rdd.filter(lambda line: "normal" not in line[-1] and line[21] !='1')\
                                .map(lambda line: (line[1], 1)).reduceByKey(lambda x,y: x+y)

normal_keyvalue = pd.DataFrame({'Label': normal_protocols_rdd.keys().collect(),
                                'State': 'normal',
                                'Count': normal_protocols_rdd.values().collect()})

attack_keyvalue = pd.DataFrame({'Label': attack_protocols_rdd.keys().collect(),
                                'State': 'normal',
                                'Count': normal_protocols_rdd.values().collect()})

results = normal_keyvalue.append(attack_keyvalue)
results.sort_values(by='Label', ascending=False)

AttributeError: 'DataFrame' object has no attribute 'append'

In [37]:
# Get a summary statistics for the sum of tcp connections to the same destination IP
from pyspark.mllib.stat import Statistics
from math import sqrt

summary = Statistics.colStats(Split_rdd.filter(lambda line: line[1] == "tcp").map(lambda line: [int(line[31])]))

tcp_mean = round(float(summary.mean()),3)
tcp_std = round(float(sqrt(summary.variance())),3)
tcp_min = round(float(summary.min()),3)
tcp_max = round(float(summary.max()),3)

print([tcp_mean, tcp_std, tcp_min, tcp_max])

[201.752, 90.726, 0.0, 255.0]


<ipython-input-37-ef8cd3912c40>:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tcp_mean = round(float(summary.mean()),3)
<ipython-input-37-ef8cd3912c40>:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tcp_std = round(float(sqrt(summary.variance())),3)
<ipython-input-37-ef8cd3912c40>:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tcp_min = round(float(summary.min()),3)
<ipython-input-37-ef8cd3912c40>:10: DeprecationWarning: Conversion of an array with ndim > 0 